# Dimensiones observables de calidad de mercado
---
En este notebook se presenta la metodología de preprocesamiento para una base de datos de la acción de Ecopetrol (ECOPETL). El procedimiento descrito estandariza la información para que esta sea utilizada para calcular distintos parámetros de calidad de mercado. 

Una vez las bases de datos hayan sido inicializadas, esta información puede ser interpretada por los métodos de visualización presentados en el notebook `Visualization.ipynb`

Este notebook está organizado de la siguiente forma:

 1. Manejo de datos
 
 2. Profundidad de mercado
 
 3. Buy-sell
 
 4. Price impact
 
 5. Ejemplo

## 1. Manejo de datos
---
En esta sección se definen dos funciones:

 * `GetDate()` - Retorna un DataFrame que contiene los días para los que tenemos datos
 * `StockPreprocessing()` - Estandariza los encabezados de la base de datos y comienza a calcular algunos de los parámetros de calidad de mercado que se quiere monitorear. Estos son:
  * Precio BID, precio ASK, precio medio y *quoted spread*.


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import dateutil.parser
from pandas import DataFrame

import warnings
warnings.filterwarnings('ignore')

import time

import dask.dataframe as dd
from dask import delayed

import os

In [ ]:
# MEJORA PARA DEFINIR LOS DIAS
# stockdata["dia"] = pd.to_datetime(data['Datetime'], format='%Y:%M:%D').dt.date

In [2]:
# Funcion que genera un DataFrame con todos los dias (sin repetir) para los cuales se tienen datos

def GetDate(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of various stocks
    
    Return:
    ------
    days: DataFrame - DataFrame of all days for which there is data
    '''
    
    days = pd.DatetimeIndex(stockdata.index).normalize()
    days = pd.DataFrame(days)
    days.index = stockdata.index
    days.columns = ['dia']
    
    return days.drop_duplicates(keep='first').dia

In [30]:
# Funcion que inicializa las columnas: 'nombre', 'date_time', 'tipo', 'precio', 'volumen',
#                                      'BID', 'ASK', 'Mid_Price', 'Quoted_Spread'

def stock_preprocessing(stockdata, stock_ticker):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of various stocks
    
    stock_ticker:
    String - Ticker of the single stock we are interested in
    
    
    Return:
    ------
    stockdata:
    DataFrame - Data of stocks with the folloeing initialized columns: 
    nombre', 'date_time', 'tipo', 'precio', 'volumen', 'BID', 'ASK', 'Mid_Price', 'Quoted_Spread'
    '''
    
    stockname = stock_ticker + " CB Equity"
    
    # Se cambian los nombres de las columnas y se elimina lo demas
    stockdata         = stockdata[['name', 'times', 'type', 'value', 'size']]
    stockdata.columns = ['nombre','date_time','tipo','precio','volumen']      
    stockdata         = stockdata[['nombre','date_time','tipo','precio','volumen']]
    
    
    # Seleccionamos los datos segun la accion y el horario que nos interesan
    stockdata        = stockdata.loc[(stockdata["nombre"] == stockname)]
    stockdata.index  = stockdata.date_time
    stockdata        = stockdata.between_time('9:30','15:55')
    stockdata['dia'] = pd.DatetimeIndex(stockdata.date_time).normalize()
    
    # DataFrame de los dias para los cuales stockdata tiene datos
    days = GetDate(stockdata)
    
    # Lista donde se guardaran los resultados de los calculos diarios
    BA = []
    
    for i in days:
        
        # Sacamos los datos para un unico dia
        stockdailydata = stockdata[stockdata.dia == str(i)]
        
        # Inicializamos columnas de precios BID y ASK
        init_values = stockdailydata.precio.values
        d           = {'BID': init_values, 'ASK': init_values}
        BA_df       = pd.DataFrame(data=d)
        
        # Calculamos los valores de los precios BID y ASK
        bid       = stockdailydata['tipo'] == 'BID'
        ask       = stockdailydata['tipo'] == 'ASK'
        BA_df.BID = np.multiply(bid.values, stockdailydata.precio.values)
        BA_df.ASK = np.multiply(ask.values, stockdailydata.precio.values)
        
        # Para los precios iguales a cero se arrastra el valor anterior
        BA_df['BID'] = BA_df['BID'].replace(to_replace = 0, method = 'ffill').values
        BA_df['ASK'] = BA_df['ASK'].replace(to_replace = 0, method = 'ffill').values
        
        # Donde el BID sea menor o igual al precio ASK se pone un NaN
        BA_df = BA_df.where(BA_df.BID <= BA_df.ASK, np.nan)
        
        # Para los precios del principio del dia, debemos poner NaN
        cols = ['BID', 'ASK']
        BA_df[cols] = BA_df[cols].replace({0:np.nan, 0:np.nan})
        
        # Con los precios BID y ASK se calcula el Mid_Price y el Quoted_Spread
        BA_df['Mid_price']     = 0.5*(BA_df['BID'].values + BA_df['ASK'].values)
        BA_df['Quoted_Spread'] = (BA_df['ASK'].values - BA_df['BID'].values)/(BA_df.Mid_price.values)
        
        # Se guarda todo en el DataFrame BA
        BA.append(BA_df)
    
    # Se unifican los valores para todos los dias
    BA        = pd.concat(BA, axis=0)
    BA.index  = stockdata.index
    stockdata = pd.concat([stockdata, BA], axis=1)
    
    # Reordenamos las columnas
    stockdata = stockdata[["date_time", "dia", "nombre", "tipo", "precio", "volumen", "ASK", "BID", 
                           "Mid_price", "Quoted_Spread"]]
    stockdata = stockdata.drop(columns=["date_time"])
            
    return stockdata

In [32]:
def StockPreprocessing(stockdata, stock_ticker):
    """
    Parameters:
    ------
    stockdata:
    DataFrame - Data of various stocks
    
    stock_tickers:
    List, string - Stock ticker strings
    """
    
    if type(stock_ticker) == list:
        # Temporary list where we will save results
        temp = []
        
        # We apply the preprocessing procedure to our list of stocks
        for ticker in stock_ticker:
            
            # We preprocess the data individually by ticker
            temp.append(stock_preprocessing(stockdata, ticker))
        
        # Concatenate the results
        data = pd.concat(temp, sort=True)
        
        # Return a dataframe sorted by timestamp
        data = data.sort_index()
        data = data[["date_time", "dia", "nombre", "tipo", "precio", "volumen", "ASK", "BID",
                     "Mid_price", "Quoted_Spread"]]
        return data
    
    else:
        return stock_preprocessing(stockdata, stock_ticker)

## 2. Profundidad de mercado
---
En esta sección utilizamos la librería Dask para paralelizar el cálculo de los siguientes parámetros: *i)* La profundidad BID; *ii)* La profundidad ASK; *iii)* La profundidad TRADE.

El cálculo de estas cantidades se hace de manera diaria. Para ello, se definen las siguientes funciones:

 * `sep_date()` - Esta función arroja una lista cuyas entradas corresponden a los datos de nuestra base para cada día.
 * `DailyDepth()` - Calcula las profundidades especificadas para cada entrada de `sep_date()`.
 * `StockDepth()` - Encapsula las dos funciones anteriores para calcular los parámetros a partir de todos los datos diarios.

In [6]:
def sep_date(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of a single stock    
    
    Return:
    ------
    daily_dfs:
    List - A list whose entries are DataFrames of data in a particular day
    '''
    
    # Obtenemos los dias para los cuales tenemos datos
    days = GetDate(stockdata)
    
    # Creamos la lista donde guardaremos los datos correspondientes a un dia
    daily_dfs = []
    
    # Separamos stockdata por dias
    for i in days:
        daily_dfs.append(stockdata.loc[stockdata["dia"] == i])
        
    return daily_dfs

In [33]:
def stock_depth(stockdata, stock_ticker):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Preprocessed data of a single stock
    
    stock_ticker:
    String - Stock's ticker
    
    Return:
    ------
    result:
    DataFrame - Results of the depth calculations
    '''
    
    # Seleccionamos los datos de la accion que nos interesa
    stockname = stock_ticker + " CB Equity"
    stockdata = stockdata.loc[stockdata["nombre"]==stockname]
    
    # Obtenemos un arreglo de los dias para los cuales tenemos datos
    days = stockdata["dia"].drop_duplicates().values
    
    # Inicializamos la lista en donde guardaremos los resultados intermedios
    temp = []
    
    # Calculamos las profundidades diarias
    for day in days:
        
        # Seleccionamos los datos del dia que estamos evaluando
        df = stockdata.loc[stockdata["dia"]==day][["tipo", "precio", "volumen"]].copy()
        
        # Primeras condiciones - Ponemos el volumen de la orden segun su tipo
        a = np.where(df['tipo'] == 'ASK', df.volumen, 0)
        b = np.where(df['tipo'] == 'BID', df.volumen, 0)
        
        # Condicion - transacciones tipo TRADE de venta
        a_t = (np.where(df['tipo'] == 'TRADE', 
                        (np.where(df['tipo'].shift(1) == 'ASK', 
                                  (np.where(df['precio'] == df['precio'].shift(1), -df.volumen, 0)), 0)), 0))
        
        # Condicion - transacciones tipo TRADE de compra
        b_t = (np.where(df['tipo'] == 'TRADE', 
                        (np.where(df['tipo'].shift(1) == 'BID', 
                                  (np.where(df['precio'] == df['precio'].shift(1), -df.volumen, 0)), 0)), 0))
        
        # Realizamos la suma acumulada de las profundidades segun las condiciones
        df['ASK_depth'] = (np.where(a_t != 0, a_t, a)).cumsum()
        df['BID_depth'] = (np.where(b_t != 0, b_t, b)).cumsum()
        df['Depth']     = df['ASK_depth'] + df['BID_depth']
        df['log_depth'] = np.log(df['Depth'].values)
        
        # Eliminamos columnas redundantes
        df = df.drop(columns=["tipo", "precio", "volumen"])
        
        # Vamos guardando los resultados de cada dia
        temp.append(df)
        
    # Concatenamos los resultados obtenidos en un unico DataFrame
    temp = pd.concat(temp, sort=True)
    
    # Concatenamos los resultados obtenidos con los datos previos
    result = pd.concat([stockdata, temp], axis=1)
    
    # date_time es el indice del DataFrame, entonces tenerlo como otra columna es redundante
    #result = result.drop(columns=["date_time"])
    
    # Reordenamos las columnas
    result = result[["dia", "nombre", "tipo", "precio", "ASK", "BID", "Mid_price", "Quoted_Spread", "volumen", 
                     "ASK_depth", "BID_depth", "Depth", "log_depth"]]
    # Retornamos el resultado
    return result

In [34]:
def StockDepth(stockdata, stock_ticker):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Preprocessed data of a several stocks
    
    stock_tickers:
    List - List of stock ticker strings
    
    Return:
    ------
    result:
    DataFrame - Results of the depth calculations
    '''
    
    #assert type(stock_tickers) == list, "To calculate a single stock's depth use the SingleStockDepth() method or pass ticker within list"
    
    if type(stock_ticker) == list:
        
        # A list where we will store intermediate results is initialized
        temp = []
        
        # We calculate each stock's depth individually
        for ticker in stock_tickers:
            
            # A stock is selected
            stockname = ticker + " CB Equity"
            df = stockdata.loc[stockdata["nombre"]==stockname]
            
            # Stock results are appended
            temp.append(SingleStockDepth(df, ticker))
    
        # Results are concatenated
        result = pd.concat(temp, sort=True)
        
        # Reordering the columns
        result = result[["dia", "nombre", "tipo", "precio", "ASK", "BID", "Mid_price", "Quoted_Spread", "volumen", 
                         "ASK_depth", "BID_depth", "Depth", "log_depth"]]
        
        return result.sort_index()
    
    else:
        
        return stock_depth(stockdata, stock_ticker)

## 3. Buy-sell
---
En esta sección calculamos la dirección de la transacción. Es decir, se calcula si la transacción es iniciada por el comprador o por el vendedor. Para ello, definimos la función `InitiatingParty()`. Este método funciona de la siguiente manera:
 * Se filtran los datos para tener únicamente transacciones tipo TRADE
 * Los precios que se ven se comparan con el precio medio:
  * Si el precio ofrecido es **mayor** que el precio medio, la transacción fue iniciada por un *comprador* ($D=+1$)
  * Si el precio ofrecido es **menor** que el precio medio, la transacción fue iniciada por un *vendedor* ($D=-1$)

In [59]:
def InitiatingParty(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Preprocessed data of the stock
    
    Return:
    ------
    x:
    DataFrame - DataFrame of TRADE quotes with the party that initiated the trade
    '''
    
    # Only TRADE operations are considered
    stockdata = stockdata.loc[stockdata["tipo"] == "TRADE"]
    
    # Values are filled according to the transaction's direction criteria
    stockdata["iniciado"] = np.where(stockdata.precio > stockdata.Mid_price, 1, 
                                     np.where(np.isnan(stockdata.Mid_price), np.nan, -1))
    
    return stockdata

## 4. Price impact
---

El *price impact* es un parámetro no-observable que cuantifica el efecto que tiene la actividad del mercado sobre el precio de un activo. En este notebook calculamos el impacto en el precio según Kyle, quien argumenta que el volumen de activos transados impacta linealmente su precio.

En este modelo, el cambio del precio de mercado por las órdenes de compra está dado por:

$$\Delta P_{i, t, d} = \beta_{i, d}^1 D_{i, t, d}V_{i, t, d} + \varepsilon_{i, t, d}$$

donde $\Delta P_{i, t, d}$ denota el cambio en el precio del activo $i$, en la transacción $t$ del día $d$. $D_{i, t, d}$ es la dirección de la transacción, es decir si la transacción fue iniciada por un comprador ($D = +1$) o por un vendedor ($D=-1$). 

$V_{i, t, d}$ es el volumen de la orden.

En esta sección calculamos el impacto de precio según Kyle 1985. 
 * `ImpactParameters()` - Calcula los parámetros necesarios para realizar la regresión de impacto de Kyle.
 * `KyleImpactRegression()` - Calcula el valor del impacto al precio.

In [8]:
from datetime import timedelta
from sklearn import linear_model as lm
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [63]:
def impact_params(stockdata):
    days = stockdata["dia"].drop_duplicates().values
    res  = []
    
    for day in days:
        stockdailydata = stockdata.loc[stockdata["dia"] == day]
        
        stockdailydata['delta_p']    = stockdailydata['precio'].diff()
        stockdailydata['order_flow'] = stockdailydata.volumen.values * stockdailydata.iniciado.values
        
        res.append(stockdailydata)
        
    res_df = pd.concat(res, axis=0)
    return res_df

In [64]:
def ImpactParameters(stockdata, stock_ticker):
    
    if(type(stock_ticker) == list):
        
        temp = []
        
        for ticker in stock_ticker:
            
            stockname = stock_ticker + " CB Equity"
            df = stockdata.loc[stockdata["nombre"]==stockname]
            temp.append(impact_params(df))
            
        result = pd.concat(temp, sort=True)
        
        return result.sort_index()
            
    else:
        stockname = stock_ticker + " CB Equity"
        df = stockdata.loc[stockdata["nombre"]==stockname]
        
        return impact_params(df)

In [10]:
def KyleImpactRegression(stockdata):
    
    days = GetDate(stockdata)#.drop_duplicates(keep='first').dia
    res = []
    
    for i in days:
        
        stockdailydata = stockdata[stockdata.dia == str(i)]
        
        x1 = stockdailydata.delta_p.values
        x1 = x1.reshape(-1, 1)
        
        x2 = stockdailydata.order_flow.values
        x2 = sm.add_constant(x2.reshape(-1, 1))
        
        result = sm.OLS(x1, x2, missing='drop').fit()
        
        coef = result.params[1]
        pvalue = result.pvalues[1]
        trades = len(stockdailydata)
        
        temp = [i, coef, pvalue, trades]
        res.append(temp)
        
    #res = pd.DataFrame(res, columns=['day', 'reg_coefficient', 'p_value', 'trades'])
    res = pd.DataFrame(res, columns=['dia', 'coef_regresion', 'p_value', 'trades'])
    res = res.set_index('dia')
    
    return res

## Ejemplo
---
Ahora aplicamos las funciones definidas en las secciones previas:

**1. Manejo de datos**

In [4]:
# Establecemos la ruta del directorio para importar los datos
pathData = os.getcwd() + "/datos completos.csv"

In [5]:
# Importamos los datos
data = %time pd.read_csv(pathData, parse_dates=[2], sep=',', na_values='NA',low_memory=False)

CPU times: user 3.18 s, sys: 228 ms, total: 3.4 s
Wall time: 3.47 s


In [18]:
# Preprocesamos los datos
prep_data = %time StockPreprocessing(data, ['ECOPETL', 'BCOLO'])

CPU times: user 2.78 s, sys: 36.1 ms, total: 2.82 s
Wall time: 2.82 s


In [19]:
prep_data.head(35)

,ASK,BID,Mid_price,Quoted_Spread,date_time,dia,nombre,precio,tipo,volumen
date_time,,,,,,,,,,
2017-03-03 09:30:00,1320.0,NaN,NaN,NaN,2017-03-03 09:30:00,2017-03-03,ECOPETL CB Equity,1320.0,ASK,360000.0
2017-03-03 09:30:00,1320.0,1305.0,1312.5,0.011429,2017-03-03 09:30:00,2017-03-03,ECOPETL CB Equity,1305.0,BID,100000.0
2017-03-03 09:30:02,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:02,2017-03-03,ECOPETL CB Equity,1315.0,ASK,80000.0
2017-03-03 09:30:03,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:03,2017-03-03,ECOPETL CB Equity,1315.0,ASK,155240.0
2017-03-03 09:30:12,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:12,2017-03-03,ECOPETL CB Equity,1305.0,BID,170000.0
2017-03-03 09:30:14,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:14,2017-03-03,ECOPETL CB Equity,1315.0,ASK,153240.0
2017-03-03 09:30:14,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:14,2017-03-03,ECOPETL CB Equity,1315.0,TRADE,2000.0
2017-03-03 09:30:22,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:22,2017-03-03,ECOPETL CB Equity,1305.0,BID,169000.0
2017-03-03 09:30:22,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:22,2017-03-03,ECOPETL CB Equity,1305.0,TRADE,1000.0


In [58]:
test = InitiatingParty(prep_data)
test.head(15)

,ASK,BID,Mid_price,Quoted_Spread,date_time,dia,nombre,precio,tipo,volumen,iniciado
date_time,,,,,,,,,,,
2017-03-03 09:30:14,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:14,2017-03-03,ECOPETL CB Equity,1315.0,TRADE,2000.0,1.0
2017-03-03 09:30:22,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:22,2017-03-03,ECOPETL CB Equity,1305.0,TRADE,1000.0,-1.0
2017-03-03 09:30:40,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:30:40,2017-03-03,ECOPETL CB Equity,1305.0,TRADE,1351.0,-1.0
2017-03-03 09:31:00,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:31:00,2017-03-03,ECOPETL CB Equity,1305.0,TRADE,500.0,-1.0
2017-03-03 09:31:00,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:31:00,2017-03-03,ECOPETL CB Equity,1305.0,TRADE,500.0,-1.0
2017-03-03 09:31:00,1315.0,1305.0,1310.0,0.007634,2017-03-03 09:31:00,2017-03-03,ECOPETL CB Equity,1305.0,TRADE,500.0,-1.0
2017-03-03 09:31:00,1310.0,1305.0,1307.5,0.003824,2017-03-03 09:31:00,2017-03-03,ECOPETL CB Equity,1305.0,TRADE,1000.0,-1.0
2017-03-03 09:31:01,1310.0,1305.0,1307.5,0.003824,2017-03-03 09:31:01,2017-03-03,ECOPETL CB Equity,1305.0,TRADE,4324.0,-1.0
2017-03-03 09:31:01,1310.0,1305.0,1307.5,0.003824,2017-03-03 09:31:01,2017-03-03,ECOPETL CB Equity,1305.0,TRADE,500.0,-1.0


**2. Profundidad de mercado**

In [219]:
# Calculamos las profundidaes BID, ASK y TRADE

res = %time StockDepth(prep_data, ["ECOPETL", "BCOLO"])

CPU times: user 2.15 s, sys: 20 ms, total: 2.17 s
Wall time: 2.17 s


In [220]:
# Miramos los resultados
res.head(12)

,dia,nombre,tipo,precio,ASK,BID,Mid_price,Quoted_Spread,volumen,ASK_depth,BID_depth,Depth,log_depth
date_time,,,,,,,,,,,,,
2017-03-03 09:30:00,2017-03-03,ECOPETL CB Equity,ASK,1320.0,1320.0,NaN,NaN,NaN,360000.0,360000.0,0.0,360000.0,12.793859
2017-03-03 09:30:00,2017-03-03,ECOPETL CB Equity,BID,1305.0,1320.0,1305.0,1312.5,0.011429,100000.0,360000.0,100000.0,460000.0,13.038982
2017-03-03 09:30:02,2017-03-03,ECOPETL CB Equity,ASK,1315.0,1315.0,1305.0,1310.0,0.007634,80000.0,440000.0,100000.0,540000.0,13.199324
2017-03-03 09:30:03,2017-03-03,ECOPETL CB Equity,ASK,1315.0,1315.0,1305.0,1310.0,0.007634,155240.0,595240.0,100000.0,695240.0,13.452012
2017-03-03 09:30:12,2017-03-03,ECOPETL CB Equity,BID,1305.0,1315.0,1305.0,1310.0,0.007634,170000.0,595240.0,270000.0,865240.0,13.670762
2017-03-03 09:30:14,2017-03-03,ECOPETL CB Equity,ASK,1315.0,1315.0,1305.0,1310.0,0.007634,153240.0,748480.0,270000.0,1018480.0,13.833822
2017-03-03 09:30:14,2017-03-03,ECOPETL CB Equity,TRADE,1315.0,1315.0,1305.0,1310.0,0.007634,2000.0,746480.0,270000.0,1016480.0,13.831856
2017-03-03 09:30:22,2017-03-03,ECOPETL CB Equity,BID,1305.0,1315.0,1305.0,1310.0,0.007634,169000.0,746480.0,439000.0,1185480.0,13.985658
2017-03-03 09:30:22,2017-03-03,ECOPETL CB Equity,TRADE,1305.0,1315.0,1305.0,1310.0,0.007634,1000.0,746480.0,438000.0,1184480.0,13.984814


In [16]:
# Guardamos los resultados
#res.to_csv("depth_data.csv")

**3. Buy-sell**

In [17]:
# Encontramos la partida que inicia la transaccion
init_trans = InitiatingParty(prep_data)

In [18]:
# Miramos los resultados
init_trans.head()

,nombre,date_time,tipo,precio,volumen,dia,BID,ASK,Mid_price,Quoted_Spread,iniciado
date_time,,,,,,,,,,,
2017-03-03 09:30:14,ECOPETL CB Equity,2017-03-03 09:30:14,TRADE,1315.0,2000.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,1
2017-03-03 09:30:22,ECOPETL CB Equity,2017-03-03 09:30:22,TRADE,1305.0,1000.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1
2017-03-03 09:30:40,ECOPETL CB Equity,2017-03-03 09:30:40,TRADE,1305.0,1351.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1
2017-03-03 09:31:00,ECOPETL CB Equity,2017-03-03 09:31:00,TRADE,1305.0,500.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1
2017-03-03 09:31:00,ECOPETL CB Equity,2017-03-03 09:31:00,TRADE,1305.0,500.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1


**4. Price impact**

In [19]:
# Calculamos los parametros de impacto
impact_params = ImpactParameters(init_trans)

# Miramos los resultados
impact_params.head()

,nombre,date_time,tipo,precio,volumen,dia,BID,ASK,Mid_price,Quoted_Spread,iniciado,delta_p,order_flow
date_time,,,,,,,,,,,,,
2017-03-03 09:30:14,ECOPETL CB Equity,2017-03-03 09:30:14,TRADE,1315.0,2000.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,1,NaN,2000.0
2017-03-03 09:30:22,ECOPETL CB Equity,2017-03-03 09:30:22,TRADE,1305.0,1000.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1,-10.0,-1000.0
2017-03-03 09:30:40,ECOPETL CB Equity,2017-03-03 09:30:40,TRADE,1305.0,1351.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1,0.0,-1351.0
2017-03-03 09:31:00,ECOPETL CB Equity,2017-03-03 09:31:00,TRADE,1305.0,500.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1,0.0,-500.0
2017-03-03 09:31:00,ECOPETL CB Equity,2017-03-03 09:31:00,TRADE,1305.0,500.0,2017-03-03,1305.0,1315.0,1310.0,0.007634,-1,0.0,-500.0


In [19]:
# Calculamos el coeficiente de impacto
kyle_reg = KyleImpactRegression(impact_params)

# Miramos los resultados
kyle_reg.head()

,coef_regresion,p_value,trades
dia,,,
2017-03-03,1.884887e-07,7.937333e-01,176
2017-03-06,7.987430e-06,7.632925e-02,277
2017-03-07,1.599549e-06,2.831874e-01,243
2017-03-08,4.147077e-06,3.137254e-02,377
2017-03-09,1.280608e-05,9.602215e-08,289
